<a href="https://colab.research.google.com/github/SeeBook2022/SeeBook-OCR/blob/main/OCR/img_preprocessing_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 환경 셋팅


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install --upgrade google-api-python-client

In [ ]:
%pip install google-cloud-vision

In [ ]:
import os

In [ ]:
!ls -l $GOOGLE_APPLICATION_CREDENTIALS
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] =  '/content/drive/MyDrive/aiffel_hackathon3/google_api_key/my_google_api_key.json'

## google api & 이미지 전처리

- resize
- contrast
- brightness
- byte화

In [ ]:
def detect_text(img_bytes):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    # with io.open(path, 'rb') as image_file:
    #     content = image_file.read()
        
    image = vision.Image(content=img_bytes) # byte로 받아서 바로 실행

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    # 제목을 한 문장으로 추출할 수 있도록 코드 수정 (공백 제거)
    result = ''
    for idx, text in enumerate(texts):
        if idx > 0: result += text.description 
    return result

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [ ]:
# 1차 전처리 함수
def img2gray(path):
    import cv2
    img = cv2.imread(path) # 이미지 불러오기
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # 흑백 처리
    add_dst = cv2.add(img_gray, 100) # 밝기 조절
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4)) # 대비 조절
    gray_cont_dst = clahe.apply(add_dst)

    img_bytes = cv2.imencode('.jpg', gray_cont_dst)[1].tobytes()

    return img_bytes

In [ ]:
# result_list = []
img_dir = '/content/drive/MyDrive/aiffel_hackathon3/test_dataset3_ver2/'
for i in range(35):
    path = img_dir + f'test_title{i+1}.jpg'
    print(f'{i+1} 번째 책:')
    print(detect_text(img2gray(path)))
    print('='*30)
    # result_list.append(detect_text(paths)) # 데이터 프레임화를 위한 리스트

In [ ]:
# 2차 전처리 함수
def img2gray2(path):
    # from google.colab.patches import cv2_imshow
    import cv2
    img = cv2.imread(path) # 이미지 불러오기
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # 흑백 처리
    img_gray = cv2.resize(img_gray, dsize=(0,0), fx = 1.5, fy= 1.5, interpolation=cv2.INTER_AREA)
    add_dst = cv2.add(img_gray, 30) # 밝기 조절
    # clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4)) # 대비 조절
    # gray_cont_dst = clahe.apply(add_dst)

    img_bytes = cv2.imencode('.jpg', add_dst)[1].tobytes()

    return img_bytes

    # cv2_imshow(add_dst)

In [ ]:
def show_img(path): # 이미지 확인 용도
    from google.colab.patches import cv2_imshow
    import cv2
    img = cv2.imread(path) # 이미지 불러오기
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # 흑백 처리
    img_gray = cv2.resize(img_gray, dsize=(0,0), fx = 1.5, fy= 1.5, interpolation=cv2.INTER_AREA) # 진우님 
    add_dst = cv2.add(img_gray, 30) # 밝기 조절

    cv2_imshow(add_dst)

    img_dir = '/content/drive/MyDrive/aiffel_hackathon3/test_dataset3_ver2/'
for i in range(5):
    path = img_dir + f'test_title{i+1}.jpg'
    show_img(path)

## 데이터 프레임화 & 결과 비교

In [ ]:
result_list = []
img_dir = '/content/drive/MyDrive/aiffel_hackathon3/test_dataset3_ver2/'
for i in range(100):
    path = img_dir + f'test_title{i+1}.jpg'
    # print(f'{i+1} 번째 책:')
    # print(detect_text(img2gray2(path)))
    # print('='*30)
    result_list.append(detect_text(img2gray2(path))) # 데이터 프레임화를 위한 리스트

In [ ]:
import pandas as pd
# 정량 평가를 위한 테스트 데이터 라벨 csv 파일 불러오기
df1 = pd.read_csv('/content/drive/MyDrive/aiffel_hackathon3/test_dataset3_title.csv', encoding = 'UTF-8')

# 테스트 데이터 라벨과 예측값 합치기
df1 = df1.assign(result = result_list)

In [ ]:
# 후처리 함수 (특수문자 및 공백 제거)
def preprocess_sentence(sentence):
    import re
    sentence = re.sub('[-=+,「#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', sentence)
    sentence = sentence.replace(' ', '') # 공백 제거
    sentence = sentence.strip()
    return sentence

In [ ]:
# 후처리 
for idx, sentence in enumerate(df1['책제목']):
    df1['책제목'][idx] = preprocess_sentence(sentence)
for idx, sentence in enumerate(df1['result']):
    df1['result'][idx] = preprocess_sentence(sentence)

# 라벨링 잘못된 데이터 수정
df1['책제목'][52] = '지금여기의페미니즘×민주주의'

In [ ]:
# 라벨, 예측값 비교 (성능 검증)
val_1st = []
for i in range(100):
    if df1['result'][i] == '':
        val_1st.append('0%')
    elif df1['책제목'][i] == df1['result'][i]:
        val_1st.append('100%')
    else:
        val_1st.append('False')

df2 = df1.assign(compare1 = val_1st)

In [ ]:
df2

,파일명,책제목,result,compare1
0,test_title1.jpg,세월이가면,세월이가면,100%
1,test_title2.jpg,님의침묵,님의침묵,100%
2,test_title3.jpg,먼후일,먼후일,100%
3,test_title4.jpg,일리아스,일리아스,100%
4,test_title5.jpg,모모,모모,100%
...,...,...,...,...
95,test_title96.jpg,수학세계사를만나다,수학세계사를만나다,100%
96,test_title97.jpg,알고리즘인생을계산하다,알고리즘인생을계산하다,100%
97,test_title98.jpg,사회적원자,사회적원자,100%
98,test_title99.jpg,우리는왜개는사랑하고돼지는먹고소는신을까,우리는왜개는사랑하고돼지는먹고소는신을까,100%


In [ ]:
df2['compare1'].value_counts()

100%     75
False    24
0%        1
Name: compare1, dtype: int64

## 유진님 결과

In [ ]:
def detect_text(path): # file path로 받아오는 거 주의 !!
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()
        
    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    # 제목을 한 문장으로 추출할 수 있도록 코드 수정 (공백 제거)
    result = ''
    for idx, text in enumerate(texts):
        if idx > 0: result += text.description 
    return result

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [ ]:
result_list2 = []
img_dir = '/content/drive/MyDrive/aiffel_hackathon3/yujin2/'
for i in range(100):
    paths = img_dir + f'gray_image_{i}.jpg'
    # print(f'{i} 번째 책:')
    # print(detect_text(paths))
    # print('='*30)
    result_list2.append(detect_text(paths)) # 데이터 프레임화를 위한 리스트

In [ ]:
import pandas as pd
# 정량 평가를 위한 테스트 데이터 라벨 csv 파일 불러오기
df = pd.read_csv('/content/drive/MyDrive/aiffel_hackathon3/test_dataset3_title.csv', encoding = 'UTF-8')

# 테스트 데이터 라벨과 예측값 합치기
df = df.assign(result = result_list2)

In [ ]:
for idx, sentence in enumerate(df['책제목']):
    df['책제목'][idx] = preprocess_sentence(sentence)
for idx, sentence in enumerate(df['result']):
    df['result'][idx] = preprocess_sentence(sentence)

# 라벨링 잘못된 데이터 수정
df['책제목'][52] = '지금여기의페미니즘×민주주의'

# 라벨, 예측값 비교 (성능 검증)
val_1st = []
for i in range(100):
    if df['result'][i] == '':
        val_1st.append('0%')
    elif df['책제목'][i] == df['result'][i]:
        val_1st.append('100%')
    else:
        val_1st.append('False')

In [ ]:
df3 = df.assign(compare1 = val_1st)
df3

,파일명,책제목,result,compare1
0,test_title1.jpg,세월이가면,세월이가면,100%
1,test_title2.jpg,님의침묵,님의침묵,100%
2,test_title3.jpg,먼후일,먼후일,100%
3,test_title4.jpg,일리아스,아,False
4,test_title5.jpg,모모,모모,100%
...,...,...,...,...
95,test_title96.jpg,수학세계사를만나다,수학세계사를만나다,100%
96,test_title97.jpg,알고리즘인생을계산하다,알고리즘인생을계산하다,100%
97,test_title98.jpg,사회적원자,사회적원자,100%
98,test_title99.jpg,우리는왜개는사랑하고돼지는먹고소는신을까,우리는왜개는사랑하고돼지는먹고소는신을까,100%


In [ ]:
df3['compare1'].value_counts()

100%     69
False    29
0%        2
Name: compare1, dtype: int64